In [1]:
import pandas as pd
import numpy as np
import dtale as dt



In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [3]:
torch.cuda.empty_cache()

In [4]:
torch.cuda.is_available()

True

In [5]:
y_train=pd.read_pickle('y_train.pkl')
y_test=pd.read_pickle('y_test.pkl')
x_test=pd.read_pickle('x_test.pkl')
x_train=pd.read_pickle('x_train.pkl')

In [6]:
pred_col='log_bearish_week'

In [7]:
test_y=y_test[pred_col]

In [8]:
training_data1=x_train

In [9]:
Y_train=y_train[pred_col]
#Y_train=df['Close']

# Data Loading


In [10]:
from torch.utils.data import Dataset, DataLoader
import joblib
class MyDataset(Dataset):
    def __init__(self, X, Y):
        #normalize

    
        self.X = torch.tensor(X, dtype=torch.float32)
        new_shape = (len(Y), 1)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        self.Y = self.Y.view(new_shape)
        
    
        
        #self.Y = self.Y.view(new_shape)
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Normalize the X data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [11]:
#scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = StandardScaler()
scaled_fit=scaler.fit_transform(training_data1)
        

joblib.dump(scaler, 'sc.joblib') 

['sc.joblib']

In [12]:
#scaler_y = StandardScaler()
scaled_fit_y=Y_train.values.reshape(-1, 1)
       

In [13]:
# Assuming X is a DataFrame with 5 columns and Y is a DataFrame with 1 column
#train_data = MyDataset(scaled_fit, Y_train.values.reshape(-1, 1))
train_data = MyDataset(scaled_fit, scaled_fit_y)

train_loader = DataLoader(train_data, shuffle=True, batch_size=32) #

In [14]:
training_data1

,log_normalized_change,log_price_range,log_price_range_high,log_price_range_low,log_premarket_changes,log_Smart_Money,log_volume_deviation,log_norm_avg_deviation_200,log_norm_avg_deviation_300,log_ha_change,log_ha_change_intra,log_ha_change_intra_high,log_ha_change_intra_low,log_ha_high_low
299,0.012928,0.013987,0.010706,-0.003281,0.003214,0.009713,0.273865,0.070582,0.106799,-0.002194,-0.010152,0.000000,-0.017736,0.017736
300,0.001720,0.008260,0.006735,-0.001525,-0.000729,0.002449,-0.231275,0.071486,0.107933,0.006601,0.001513,0.006329,-0.001932,0.008260
301,-0.010966,0.013219,0.005758,-0.007461,-0.004173,-0.006793,0.517538,0.059751,0.096423,-0.005754,-0.004997,0.002870,-0.010349,0.013219
302,0.014857,0.015574,0.010378,-0.005196,0.005664,0.009193,0.296428,0.073756,0.110679,0.004595,0.002093,0.008857,-0.006718,0.015574
303,0.006366,0.007144,0.006947,-0.000197,0.001119,0.005247,0.039677,0.079242,0.116441,0.009702,0.010748,0.014690,0.000000,0.014690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4262,-0.015447,0.017550,0.000618,-0.016932,-0.000124,-0.015324,0.208525,0.018386,-0.017319,-0.009913,-0.002042,0.006452,-0.011098,0.017550
4263,0.001631,0.007836,0.003308,-0.004528,0.000301,0.001329,-0.173034,0.019831,-0.015107,-0.007116,-0.008137,0.000000,-0.012697,0.012697
4264,-0.018622,0.027653,0.004343,-0.023310,0.002053,-0.020676,0.202728,0.001120,-0.033089,-0.006484,-0.010561,0.003618,-0.024035,0.027653
4265,-0.014535,0.022741,0.005535,-0.017206,-0.001457,-0.013078,0.651889,-0.013339,-0.046932,-0.018441,-0.023735,0.000000,-0.034797,0.034797


# Pytorch setup

In [15]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, num_heads):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_heads = num_heads

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Reshape the input tensor to match the expected shape of (batch_size, sequence_length, input_size)
        x = x.view(-1, 1, x.shape[-1])
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        output, (h_n, c_n) = self.lstm(x)
        output = output.permute(1, 0, 2)  # (seq_len, batch_size, hidden_size)
        attention_output, _ = self.attention(output, output, output)  # apply self-attention
        output = output + attention_output  # add residual connection
        output = output.permute(1, 0, 2)  # (batch_size, seq_len, hidden_size)
        output = self.relu(output[:, -1, :])
        output = self.dropout(output)
        output = self.fc(output)
        output = self.sigmoid(output)

        return torch.squeeze(output)


In [16]:
num_epochs = 35000
learning_rate = 0.0005
num_layers=5
input_size = len(x_train.columns)
hidden_size = 90
output_size = 1
dropout=.5
num_heads=5

In [17]:
def array_list_to_dataframe(arrays,length):
    # Convert the list of arrays to a single 2D array
    array_2d = np.concatenate(arrays, axis=1)
    
    # Reshape the 2D array into a 5-column array
    array_5col = np.reshape(array_2d, (-1, length))
    
    # Create a DataFrame from the 5-column array
    df = pd.DataFrame(array_5col)
    
    return df

In [18]:
certainty_2sd_counts=pd.DataFrame()
certainty_4sd_counts=pd.DataFrame()
certainty_5sd_counts=pd.DataFrame()
certainty_6sd_counts=pd.DataFrame()


In [19]:
#set up gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
for j in range(7):
    # Instantiate the model
    model = LSTM(input_size=input_size, hidden_size=hidden_size,num_layers=num_layers,output_size=1,dropout=dropout,num_heads=num_heads)#output_size=1
    model=torch.compile(model)
    model.to(device)
    train_data_x=train_data.X.to(device)
    train_data_y=train_data.Y.to(device)
    # Define the loss function and optimizer
    criterion = nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=0.003,weight_decay=6e-7) 
        # Train the model
    for epoch in range(num_epochs):
        outputs = model(train_data_x)
        loss = criterion(outputs, train_data_y.squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1000 == 0:
            print("Epoch: %d, loss: %1.5f"  % (epoch, loss.item()),j)
        """for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y.squeeze())
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))"""
        
        
    #Transform test data
    transformedData = scaler.transform(x_test)
    transformedData_y=test_y.values.reshape(-1, 1)

    # Use the trained model for predictions
    
    
    
    test_data = MyDataset(transformedData, transformedData_y)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
    test_data_x=test_data.X.to(device)
    
    outputlist=[]
    
    #testing data
    
    model.train()
    len_outputs=100
    with torch.no_grad():
        for i in range(len_outputs):
            output = model(test_data_x)
            outputlist.append(output)
    model.eval()
    
    scaled_output_list=[]
    for outputs in outputlist:
        scaled_output=outputs.cpu().reshape(-1, 1)
        scaled_output_list.append(scaled_output)
        
    model_test_outputs=array_list_to_dataframe(scaled_output_list,len_outputs )
    
    count1 = (model_test_outputs > .999).sum(axis=1)
    count2=(model_test_outputs > .9999).sum(axis=1)
    count3=(model_test_outputs > .99999).sum(axis=1)
    count4=(model_test_outputs > .999999).sum(axis=1)
    
    
    column_name = 'Column ' + str(j)
    
    certainty_2sd_counts[column_name] =count1 
    certainty_4sd_counts[column_name] =count2
    certainty_5sd_counts[column_name] =count3
    certainty_6sd_counts[column_name] =count4

/home/jeffwa/anaconda3/envs/DL_new/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



Epoch: 0, loss: 0.72650 0
Epoch: 1000, loss: 0.22662 0
Epoch: 2000, loss: 0.11121 0
Epoch: 3000, loss: 0.00007 0
Epoch: 4000, loss: 0.00003 0
Epoch: 5000, loss: 0.00001 0
Epoch: 6000, loss: 0.02484 0
Epoch: 7000, loss: 0.00004 0
Epoch: 8000, loss: 0.00014 0
Epoch: 9000, loss: 0.00002 0
Epoch: 10000, loss: 0.11895 0
Epoch: 11000, loss: 0.00013 0
Epoch: 12000, loss: 0.00004 0
Epoch: 13000, loss: 0.00003 0
Epoch: 14000, loss: 0.00001 0
Epoch: 15000, loss: 0.00001 0
Epoch: 16000, loss: 0.00001 0
Epoch: 17000, loss: 0.00001 0
Epoch: 18000, loss: 0.00000 0
Epoch: 19000, loss: 0.00005 0
Epoch: 20000, loss: 0.00003 0
Epoch: 21000, loss: 0.00001 0
Epoch: 22000, loss: 0.00002 0
Epoch: 23000, loss: 0.00002 0
Epoch: 24000, loss: 0.00001 0
Epoch: 25000, loss: 0.02522 0
Epoch: 26000, loss: 0.02521 0
Epoch: 27000, loss: 0.00001 0
Epoch: 28000, loss: 0.00001 0
Epoch: 29000, loss: 0.00001 0
Epoch: 30000, loss: 0.00002 0
Epoch: 31000, loss: 0.00001 0
Epoch: 32000, loss: 0.00004 0
Epoch: 33000, loss: 0.0

In [24]:
certainty_4sd_counts.mean(axis=1)


0     10.571429
1     44.285714
2     29.000000
3     41.714286
4      0.000000
5     12.714286
6      0.142857
7      0.000000
8     42.571429
9     23.428571
10    45.428571
11    68.285714
dtype: float64

In [23]:
certainty_4sd_counts

,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6
0,74,0,0,0,0,0,0
1,0,25,100,98,0,84,3
2,80,0,0,39,0,84,0
3,0,5,0,98,0,89,100
4,0,0,0,0,0,0,0
5,89,0,0,0,0,0,0
6,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,96,97,0,5,0,100
9,0,98,0,0,0,66,0
